Importações necessárias

In [1]:
import time

Criando o banco de dados que será utilizado

In [2]:
spark.sql("CREATE DATABASE IF NOT EXISTS earth")

DataFrame[]

Dropando tabelas que serão criadas caso elas existam

In [3]:
%%sql
DROP TABLE IF EXISTS earth.climate_change_partitioned

24/04/25 20:09:26 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [4]:
%%sql
DROP TABLE IF EXISTS earth.climate_change

++
||
++
++

Criando tabelas no formato Iceberg

Tabela particionada por país

In [5]:
df = spark.read.parquet("/home/iceberg/data/climate_change.parquet")
df.writeTo("icelake.earth.climate_change_partitioned").partitionedBy("Country").createOrReplace()

Tabela normal

In [6]:
df.writeTo("earth.climate_change").createOrReplace()

Checando o país que possui o maior número de registros

In [7]:
spark.sql("SELECT Country, COUNT(1) FROM earth.climate_change GROUP BY 1 ORDER BY 2 DESC LIMIT 5").show()

+-------------+--------+
|      Country|count(1)|
+-------------+--------+
|        India| 1014906|
|        China|  827802|
|United States|  687289|
|       Brazil|  475580|
|       Russia|  461234|
+-------------+--------+



Demonstração do tempo de execução de consultas no ambiente normal e particionado 

Tabela sem partição

In [8]:
inicio = time.time()
normal_table = spark.read.format("iceberg").load("icelake.earth.climate_change")
filtered_data = normal_table.where(normal_table.Country == "India")
fim = time.time()
print("Tempo de execução: ", fim-inicio)

Tempo de execução:  0.11427855491638184


Tabela com partição

In [9]:
inicio = time.time()
part_table = spark.read.format("iceberg").load("icelake.earth.climate_change_partitioned")
filtered_data = part_table.where(part_table.Country == "India")
fim = time.time()
print("Tempo de execução: ", fim-inicio)

Tempo de execução:  0.0909261703491211
